## Grid Search

In [14]:
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
import lightgbm as lgb

In [2]:
dataset = fetch_california_housing()

In [5]:
print(dataset['DESCR'])

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

In [10]:
X = pd.DataFrame(dataset['data'], columns=dataset['feature_names'])
y = dataset['target']

In [18]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)
# モデル
lgbmr = lgb.LGBMRegressor(random_state=0)
param_grid = {'num_leaves': [10, 20, 30, 40, 50, 60],
              'max_depth': [5, 10, 15],
              'reg_alpha': [0, 0.01, 0.03]}
cv = KFold(n_splits=3, random_state=0, shuffle=True)
gs = GridSearchCV(lgbmr, param_grid=param_grid, cv=cv)

In [30]:
# early_stopping
callbacks = [lgb.early_stopping(stopping_rounds=10)]
eval_set = [(X_val, y_val)]
fit_params = {'callbacks': callbacks, 'eval_set': eval_set}
gs.fit(X_train, y_train, **fit_params)

Training until validation scores don't improve for 10 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.264841
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.256456
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.256456
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.256456
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.256456
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.256456
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.256456
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.255538
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.255538
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.240402
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.240164
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.240164
Did not meet ea

GridSearchCV(cv=KFold(n_splits=3, random_state=0, shuffle=True),
             estimator=LGBMRegressor(random_state=0),
             param_grid={'max_depth': [5, 10, 15],
                         'num_leaves': [10, 20, 30, 40, 50, 60],
                         'reg_alpha': [0, 0.01, 0.03]})

In [21]:
cv_results_df = pd.DataFrame(gs.cv_results_)
cv_results_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_num_leaves,param_reg_alpha,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.223933,0.225003,0.013757,0.000292,5,10,0,"{'max_depth': 5, 'num_leaves': 10, 'reg_alpha'...",0.797096,0.807785,0.805461,0.803448,0.004590,54
1,0.072422,0.008640,0.013764,0.000347,5,10,0.01,"{'max_depth': 5, 'num_leaves': 10, 'reg_alpha'...",0.798894,0.806981,0.806050,0.803975,0.003613,53
2,0.074994,0.003492,0.013673,0.000204,5,10,0.03,"{'max_depth': 5, 'num_leaves': 10, 'reg_alpha'...",0.799277,0.806364,0.806658,0.804100,0.003413,52
3,0.085039,0.010063,0.017791,0.001822,5,20,0,"{'max_depth': 5, 'num_leaves': 20, 'reg_alpha'...",0.811435,0.821243,0.815893,0.816190,0.004010,36
4,0.103134,0.017353,0.017834,0.001631,5,20,0.01,"{'max_depth': 5, 'num_leaves': 20, 'reg_alpha'...",0.810028,0.821356,0.813901,0.815095,0.004701,45


In [23]:
for index, row in cv_results_df[cv_results_df['rank_test_score']<=5].iterrows():
    print(f'{row["rank_test_score"]}: {row["params"]}')
    print(f'{row["mean_test_score"]}')

4: {'max_depth': 10, 'num_leaves': 50, 'reg_alpha': 0.01}
0.8329725903236045
3: {'max_depth': 15, 'num_leaves': 50, 'reg_alpha': 0}
0.8331058990226472
2: {'max_depth': 15, 'num_leaves': 50, 'reg_alpha': 0.03}
0.8332059773155042
5: {'max_depth': 15, 'num_leaves': 60, 'reg_alpha': 0}
0.8327661059213919
1: {'max_depth': 15, 'num_leaves': 60, 'reg_alpha': 0.01}
0.8338051795879502


In [31]:
lgbmr = lgb.LGBMRegressor(**gs.best_params_, random_state=0, learning_rate=0.01, n_estimators=1000)
callbacks = [lgb.early_stopping(stopping_rounds=10)]
eval_set = [(X_val, y_val)]
fit_params = {'callbacks': callbacks, 'eval_set': eval_set}
lgbmr.fit(X_train, y_train, **fit_params)

Training until validation scores don't improve for 10 rounds
Did not meet early stopping. Best iteration is:
[1000]	valid_0's l2: 0.195693


LGBMRegressor(learning_rate=0.01, max_depth=15, n_estimators=1000,
              num_leaves=60, random_state=0, reg_alpha=0.01)

In [32]:
lgbmr.score(X_val, y_val)

0.8499232425233268